In [1]:
from keras.layers import (BatchNormalization, Convolution2D, Dense,
                          Dropout, Flatten, MaxPooling2D, Input, Add,
                          AveragePooling2D, Activation, ZeroPadding2D, 
                          Concatenate, Reshape)
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import to_categorical, plot_model
from keras.losses import sparse_categorical_crossentropy as scc

from pool_helper import PoolHelper
from lrn import LRN

import numpy as np

epochs = 10

# Simple convolutional neural network

First we'll implement simple cnn with one convolutional layer 

In [16]:
class SimpleCNN(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Convolution2D(10, kernel_size=3, activation="relu", input_shape=input_shape))
        self.add(Convolution2D(32, kernel_size=3, activation="relu"))
        self.add(Flatten())
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# VGG16 i VGG19

In [5]:
class VGG(Sequential):
    def __init__(self, input_shape, num_classes, vgg_type=16):
        super().__init__()

        self.add(Convolution2D(64, kernel_size=(3,3), padding="same", activation="relu", input_shape=input_shape))
        self.add(Convolution2D(64, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(MaxPooling2D((2, 2)))
        
        self.add(Convolution2D(128, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(Convolution2D(128, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(MaxPooling2D((2, 2)))
        
        if vgg_type == 16:
            self.__tri_conv(256)
            self.__tri_conv(512)
            self.__tri_conv(512)
        else:
            self.__qua_conv(256)
            self.__qua_conv(512)
            self.__qua_conv(512)
        
        self.add(Flatten())
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 

    def __tri_conv(self, filters):
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="valid", activation="relu"))
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(MaxPooling2D((2, 2), padding="same"))
        
                 
    def __qua_conv(self, filters):
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(Convolution2D(filters, kernel_size=(3,3), padding="same", activation="relu"))
        self.add(MaxPooling2D((2, 2), padding="same"))

# AlexNet

In [3]:
class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Convolution2D(96, kernel_size = (11,11), strides = 4,
                        padding = 'valid', activation = 'relu',
                        input_shape = input_shape,
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

        self.add(Convolution2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
                        
        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None)) 

        self.add(Convolution2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Convolution2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(Convolution2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

        self.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'same', data_format= None))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dense(1000, activation= 'relu'))
        self.add(Dense(num_classes, activation= 'softmax'))

        self.compile(optimizer= Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# SqueezeNet

In [31]:
class SqueezeNet():
    def __init__(self, input_shape, num_classes):

        InputImages = Input(shape=input_shape)

        conv1 = Convolution2D(96, (7, 7), activation='relu')(InputImages)
        maxpool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv1)

        fire2 = self.__fire(maxpool1, squeeze=16, expand=64)
        fire3 = self.__fire(fire2, squeeze=16, expand=64)
        fire4 = self.__fire(fire3, squeeze=32, expand=128)

        maxpool4 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(fire4)

        fire5 = self.__fire(maxpool4, squeeze=32, expand=128)
        fire6 = self.__fire(fire5, squeeze=32, expand=128)
        fire7 = self.__fire(fire6, squeeze=48, expand=192)
        fire8 = self.__fire(fire7, squeeze=64, expand=256)

        maxpool8 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(fire8)

        fire9 = self.__fire(maxpool8, squeeze=64, expand=256)
        fire9_dropout = Dropout(0.5)(fire9)

        conv10 = Convolution2D(10, (1,1), activation='relu', padding='same')(fire9_dropout)
        avg_pooling10 = AveragePooling2D(padding='same')(conv10)
        flatten = Flatten()(avg_pooling10)
        dense = Dense(num_classes, activation="softmax")(flatten)

        squeezeNet_model = Model(inputs=InputImages, outputs=dense)
        squeezeNet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        self.model = squeezeNet_model
        
        
    
    @staticmethod
    def __fire(X, squeeze=16, expand=64):
        fire_squeeze = Convolution2D(squeeze, (1,1), activation='relu', padding='same')(X)
        fire_expand1 = Convolution2D(expand, (1,1), activation='relu', padding='same')(fire_squeeze)
        fire_expand2 = Convolution2D(expand, (3,3), activation='relu', padding='same')(fire_squeeze)
        merge = Add()([fire_expand1, fire_expand2])
        return merge

# ResNet

In [20]:
class ResNet():
    def __init__(self, input_shape, num_classes):
        
        InputImages = Input(shape=input_shape)
        zeroPad1 = ZeroPadding2D((1,1), data_format="channels_first")
        zeroPad1_2 = ZeroPadding2D((1,1), data_format="channels_first")
        
        layer1 = Convolution2D(6, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', data_format="channels_first")
        layer1_2 = Convolution2D(16, (3, 3), strides=(2, 2), kernel_initializer='he_uniform', data_format="channels_first")
        zeroPad2 = ZeroPadding2D((1,1), data_format="channels_first")
        zeroPad2_2 = ZeroPadding2D((1,1), data_format="channels_first")
        
        layer2 = Convolution2D(6, (3, 3), strides=(1,1), kernel_initializer='he_uniform', data_format="channels_first")
        layer2_2 = Convolution2D(16, (3, 3), strides=(1,1), kernel_initializer='he_uniform', data_format="channels_first")
        zeroPad3 = ZeroPadding2D((1,1), data_format="channels_first")
        zeroPad3_2 = ZeroPadding2D((1,1), data_format="channels_first")
        
        layer3 = Convolution2D(6, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', data_format="channels_first")
        layer3_2 = Convolution2D(16, (3, 3), strides=(1, 1), kernel_initializer='he_uniform', data_format="channels_first")
        layer4 = Dense(64, activation='relu', kernel_initializer='he_uniform')
        layer5 = Dense(16, activation='relu', kernel_initializer='he_uniform')
        final = Dense(num_classes, activation='softmax', kernel_initializer='he_uniform')
        
        first = zeroPad1(InputImages)
        second = layer1(first)
        second = BatchNormalization(axis=1)(second)
        second = Activation('relu')(second)

        third = zeroPad2(second)
        third = layer2(third)
        third = BatchNormalization(axis=1)(third)
        third = Activation('relu')(third)

        third = zeroPad3(third)
        third = layer3(third)
        third = BatchNormalization(axis=1)(third)
        third = Activation('relu')(third)
        
        res = Add()([third, second])


        first2 = zeroPad1_2(res)
        second2 = layer1_2(first2)
        second2 = BatchNormalization(axis=1)(second2)
        second2 = Activation('relu')(second2)


        third2 = zeroPad2_2(second2)
        third2 = layer2_2(third2)
        third2 = BatchNormalization(axis=1)(third2)
        third2 = Activation('relu')(third2)

        third2 = zeroPad3_2(third2)
        third2 = layer3_2(third2)
        third2 = BatchNormalization(axis=1)(third2)
        third2 = Activation('relu')(third2)
        
        res2 = Add()([third2, second2])

        res2 = Flatten()(res2)

        res2 = layer4(res2)
        res2 = Dropout(0.4)(res2)
        res2 = layer5(res2)
        res2 = Dropout(0.4)(res2)
        res2 = final(res2)
        model = Model(inputs=InputImages, outputs=res2)

        sgd = SGD(decay=0., learning_rate=0.01, momentum=0.9, nesterov=True)
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
        self.model = model

# GoogleLeNet

In [14]:
class GoogleLeNet():
    def __init__(self, input_shape, num_classes):
        InputImages = Input(shape=input_shape)

        input_pad = ZeroPadding2D(padding=(3, 3))(InputImages)
        conv1_7x7_s2 = Convolution2D(64, (7,7), strides=(2,2), padding='valid', activation='relu', name='conv1/7x7_s2', kernel_regularizer=l2(0.0002))(input_pad)
        conv1_zero_pad = ZeroPadding2D(padding=(1, 1))(conv1_7x7_s2)
        pool1_helper = PoolHelper()(conv1_zero_pad)
        pool1_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool1/3x3_s2')(pool1_helper)
        pool1_norm1 = LRN(name='pool1/norm1')(pool1_3x3_s2)

        conv2_3x3_reduce = Convolution2D(64, (1,1), padding='same', activation='relu', name='conv2/3x3_reduce', kernel_regularizer=l2(0.0002))(pool1_norm1)
        conv2_3x3 = Convolution2D(192, (3,3), padding='same', activation='relu', name='conv2/3x3', kernel_regularizer=l2(0.0002))(conv2_3x3_reduce)
        conv2_norm2 = LRN(name='conv2/norm2')(conv2_3x3)
        conv2_zero_pad = ZeroPadding2D(padding=(1, 1))(conv2_norm2)
        pool2_helper = PoolHelper()(conv2_zero_pad)
        pool2_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool2/3x3_s2')(pool2_helper)

        inception_3a_1x1 = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_3a/1x1', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
        inception_3a_3x3_reduce = Convolution2D(96, (1,1), padding='same', activation='relu', name='inception_3a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
        inception_3a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3_reduce)
        inception_3a_3x3 = Convolution2D(128, (3,3), padding='valid', activation='relu', name='inception_3a/3x3', kernel_regularizer=l2(0.0002))(inception_3a_3x3_pad)
        inception_3a_5x5_reduce = Convolution2D(16, (1,1), padding='same', activation='relu', name='inception_3a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
        inception_3a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5_reduce)
        inception_3a_5x5 = Convolution2D(32, (5,5), padding='valid', activation='relu', name='inception_3a/5x5', kernel_regularizer=l2(0.0002))(inception_3a_5x5_pad)
        inception_3a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_3a/pool')(pool2_3x3_s2)
        inception_3a_pool_proj = Convolution2D(32, (1,1), padding='same', activation='relu', name='inception_3a/pool_proj', kernel_regularizer=l2(0.0002))(inception_3a_pool)
        inception_3a_output = Concatenate(axis=2, name='inception_3a/output')([inception_3a_1x1,inception_3a_3x3,inception_3a_5x5,inception_3a_pool_proj])

        inception_3b_1x1 = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_3b/1x1', kernel_regularizer=l2(0.0002))(inception_3a_output)
        inception_3b_3x3_reduce = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_3b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)
        inception_3b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3_reduce)
        inception_3b_3x3 = Convolution2D(192, (3,3), padding='valid', activation='relu', name='inception_3b/3x3', kernel_regularizer=l2(0.0002))(inception_3b_3x3_pad)
        inception_3b_5x5_reduce = Convolution2D(32, (1,1), padding='same', activation='relu', name='inception_3b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)
        inception_3b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5_reduce)
        inception_3b_5x5 = Convolution2D(96, (5,5), padding='valid', activation='relu', name='inception_3b/5x5', kernel_regularizer=l2(0.0002))(inception_3b_5x5_pad)
        inception_3b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_3b/pool')(inception_3a_output)
        inception_3b_pool_proj = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_3b/pool_proj', kernel_regularizer=l2(0.0002))(inception_3b_pool)
        inception_3b_output = Concatenate(axis=1, name='inception_3b/output')([inception_3b_1x1,inception_3b_3x3,inception_3b_5x5,inception_3b_pool_proj])
        
        inception_3b_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_output)
        pool3_helper = PoolHelper()(inception_3b_output_zero_pad)
        pool3_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool3/3x3_s2')(pool3_helper)

        inception_4a_1x1 = Convolution2D(192, (1,1), padding='same', activation='relu', name='inception_4a/1x1', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
        inception_4a_3x3_reduce = Convolution2D(96, (1,1), padding='same', activation='relu', name='inception_4a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
        inception_4a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4a_3x3_reduce)
        inception_4a_3x3 = Convolution2D(208, (3,3), padding='valid', activation='relu', name='inception_4a/3x3' ,kernel_regularizer=l2(0.0002))(inception_4a_3x3_pad)
        inception_4a_5x5_reduce = Convolution2D(16, (1,1), padding='same', activation='relu', name='inception_4a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
        inception_4a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4a_5x5_reduce)
        inception_4a_5x5 = Convolution2D(48, (5,5), padding='valid', activation='relu', name='inception_4a/5x5', kernel_regularizer=l2(0.0002))(inception_4a_5x5_pad)
        inception_4a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4a/pool')(pool3_3x3_s2)
        inception_4a_pool_proj = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_4a/pool_proj', kernel_regularizer=l2(0.0002))(inception_4a_pool)
        inception_4a_output = Concatenate(axis=1, name='inception_4a/output')([inception_4a_1x1,inception_4a_3x3,inception_4a_5x5,inception_4a_pool_proj])

        loss1_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss1/ave_pool')(inception_4a_output)
        loss1_conv = Convolution2D(128, (1,1), padding='same', activation='relu', name='loss1/conv', kernel_regularizer=l2(0.0002))(loss1_ave_pool)
        loss1_flat = Flatten()(loss1_conv)
        loss1_fc = Dense(1024, activation='relu', name='loss1/fc', kernel_regularizer=l2(0.0002))(loss1_flat)
        loss1_drop_fc = Dropout(rate=0.7)(loss1_fc)
        loss1_classifier = Dense(1000, name='loss1/classifier', kernel_regularizer=l2(0.0002))(loss1_drop_fc)
        loss1_classifier_act = Activation('softmax')(loss1_classifier)

        inception_4b_1x1 = Convolution2D(160, (1,1), padding='same', activation='relu', name='inception_4b/1x1', kernel_regularizer=l2(0.0002))(inception_4a_output)
        inception_4b_3x3_reduce = Convolution2D(112, (1,1), padding='same', activation='relu', name='inception_4b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
        inception_4b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4b_3x3_reduce)
        inception_4b_3x3 = Convolution2D(224, (3,3), padding='valid', activation='relu', name='inception_4b/3x3', kernel_regularizer=l2(0.0002))(inception_4b_3x3_pad)
        inception_4b_5x5_reduce = Convolution2D(24, (1,1), padding='same', activation='relu', name='inception_4b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
        inception_4b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4b_5x5_reduce)
        inception_4b_5x5 = Convolution2D(64, (5,5), padding='valid', activation='relu', name='inception_4b/5x5', kernel_regularizer=l2(0.0002))(inception_4b_5x5_pad)
        inception_4b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4b/pool')(inception_4a_output)
        inception_4b_pool_proj = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_4b/pool_proj', kernel_regularizer=l2(0.0002))(inception_4b_pool)
        inception_4b_output = Concatenate(axis=1, name='inception_4b/output')([inception_4b_1x1,inception_4b_3x3,inception_4b_5x5,inception_4b_pool_proj])

        inception_4c_1x1 = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_4c/1x1', kernel_regularizer=l2(0.0002))(inception_4b_output)
        inception_4c_3x3_reduce = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_4c/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
        inception_4c_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4c_3x3_reduce)
        inception_4c_3x3 = Convolution2D(256, (3,3), padding='valid', activation='relu', name='inception_4c/3x3', kernel_regularizer=l2(0.0002))(inception_4c_3x3_pad)
        inception_4c_5x5_reduce = Convolution2D(24, (1,1), padding='same', activation='relu', name='inception_4c/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
        inception_4c_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4c_5x5_reduce)
        inception_4c_5x5 = Convolution2D(64, (5,5), padding='valid', activation='relu', name='inception_4c/5x5', kernel_regularizer=l2(0.0002))(inception_4c_5x5_pad)
        inception_4c_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4c/pool')(inception_4b_output)
        inception_4c_pool_proj = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_4c/pool_proj', kernel_regularizer=l2(0.0002))(inception_4c_pool)
        inception_4c_output = Concatenate(axis=1, name='inception_4c/output')([inception_4c_1x1,inception_4c_3x3,inception_4c_5x5,inception_4c_pool_proj])

        inception_4d_1x1 = Convolution2D(112, (1,1), padding='same', activation='relu', name='inception_4d/1x1', kernel_regularizer=l2(0.0002))(inception_4c_output)
        inception_4d_3x3_reduce = Convolution2D(144, (1,1), padding='same', activation='relu', name='inception_4d/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
        inception_4d_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4d_3x3_reduce)
        inception_4d_3x3 = Convolution2D(288, (3,3), padding='valid', activation='relu', name='inception_4d/3x3', kernel_regularizer=l2(0.0002))(inception_4d_3x3_pad)
        inception_4d_5x5_reduce = Convolution2D(32, (1,1), padding='same', activation='relu', name='inception_4d/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
        inception_4d_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4d_5x5_reduce)
        inception_4d_5x5 = Convolution2D(64, (5,5), padding='valid', activation='relu', name='inception_4d/5x5', kernel_regularizer=l2(0.0002))(inception_4d_5x5_pad)
        inception_4d_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4d/pool')(inception_4c_output)
        inception_4d_pool_proj = Convolution2D(64, (1,1), padding='same', activation='relu', name='inception_4d/pool_proj', kernel_regularizer=l2(0.0002))(inception_4d_pool)
        inception_4d_output = Concatenate(axis=1, name='inception_4d/output')([inception_4d_1x1,inception_4d_3x3,inception_4d_5x5,inception_4d_pool_proj])

        loss2_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss2/ave_pool')(inception_4d_output)
        loss2_conv = Convolution2D(128, (1,1), padding='same', activation='relu', name='loss2/conv', kernel_regularizer=l2(0.0002))(loss2_ave_pool)
        loss2_flat = Flatten()(loss2_conv)
        loss2_fc = Dense(1024, activation='relu', name='loss2/fc', kernel_regularizer=l2(0.0002))(loss2_flat)
        loss2_drop_fc = Dropout(rate=0.7)(loss2_fc)
        loss2_classifier = Dense(1000, name='loss2/classifier', kernel_regularizer=l2(0.0002))(loss2_drop_fc)
        loss2_classifier_act = Activation('softmax')(loss2_classifier)

        inception_4e_1x1 = Convolution2D(256, (1,1), padding='same', activation='relu', name='inception_4e/1x1', kernel_regularizer=l2(0.0002))(inception_4d_output)
        inception_4e_3x3_reduce = Convolution2D(160, (1,1), padding='same', activation='relu', name='inception_4e/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
        inception_4e_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_3x3_reduce)
        inception_4e_3x3 = Convolution2D(320, (3,3), padding='valid', activation='relu', name='inception_4e/3x3', kernel_regularizer=l2(0.0002))(inception_4e_3x3_pad)
        inception_4e_5x5_reduce = Convolution2D(32, (1,1), padding='same', activation='relu', name='inception_4e/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
        inception_4e_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4e_5x5_reduce)
        inception_4e_5x5 = Convolution2D(128, (5,5), padding='valid', activation='relu', name='inception_4e/5x5', kernel_regularizer=l2(0.0002))(inception_4e_5x5_pad)
        inception_4e_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_4e/pool')(inception_4d_output)
        inception_4e_pool_proj = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_4e/pool_proj', kernel_regularizer=l2(0.0002))(inception_4e_pool)
        inception_4e_output = Concatenate(axis=1, name='inception_4e/output')([inception_4e_1x1,inception_4e_3x3,inception_4e_5x5,inception_4e_pool_proj])

        inception_4e_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_output)
        pool4_helper = PoolHelper()(inception_4e_output_zero_pad)
        pool4_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool4/3x3_s2')(pool4_helper)

        inception_5a_1x1 = Convolution2D(256, (1,1), padding='same', activation='relu', name='inception_5a/1x1', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
        inception_5a_3x3_reduce = Convolution2D(160, (1,1), padding='same', activation='relu', name='inception_5a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
        inception_5a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5a_3x3_reduce)
        inception_5a_3x3 = Convolution2D(320, (3,3), padding='valid', activation='relu', name='inception_5a/3x3', kernel_regularizer=l2(0.0002))(inception_5a_3x3_pad)
        inception_5a_5x5_reduce = Convolution2D(32, (1,1), padding='same', activation='relu', name='inception_5a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
        inception_5a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5a_5x5_reduce)
        inception_5a_5x5 = Convolution2D(128, (5,5), padding='valid', activation='relu', name='inception_5a/5x5', kernel_regularizer=l2(0.0002))(inception_5a_5x5_pad)
        inception_5a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_5a/pool')(pool4_3x3_s2)
        inception_5a_pool_proj = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_5a/pool_proj', kernel_regularizer=l2(0.0002))(inception_5a_pool)
        inception_5a_output = Concatenate(axis=1, name='inception_5a/output')([inception_5a_1x1,inception_5a_3x3,inception_5a_5x5,inception_5a_pool_proj])

        inception_5b_1x1 = Convolution2D(384, (1,1), padding='same', activation='relu', name='inception_5b/1x1', kernel_regularizer=l2(0.0002))(inception_5a_output)
        inception_5b_3x3_reduce = Convolution2D(192, (1,1), padding='same', activation='relu', name='inception_5b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
        inception_5b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5b_3x3_reduce)
        inception_5b_3x3 = Convolution2D(384, (3,3), padding='valid', activation='relu', name='inception_5b/3x3', kernel_regularizer=l2(0.0002))(inception_5b_3x3_pad)
        inception_5b_5x5_reduce = Convolution2D(48, (1,1), padding='same', activation='relu', name='inception_5b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
        inception_5b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5b_5x5_reduce)
        inception_5b_5x5 = Convolution2D(128, (5,5), padding='valid', activation='relu', name='inception_5b/5x5', kernel_regularizer=l2(0.0002))(inception_5b_5x5_pad)
        inception_5b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', name='inception_5b/pool')(inception_5a_output)
        inception_5b_pool_proj = Convolution2D(128, (1,1), padding='same', activation='relu', name='inception_5b/pool_proj', kernel_regularizer=l2(0.0002))(inception_5b_pool)
        inception_5b_output = Concatenate(axis=1, name='inception_5b/output')([inception_5b_1x1,inception_5b_3x3,inception_5b_5x5,inception_5b_pool_proj])

        pool5_7x7_s1 = AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
        loss3_flat = Flatten()(pool5_7x7_s1)
        pool5_drop_7x7_s1 = Dropout(rate=0.4)(loss3_flat)
        loss3_classifier = Dense(1000, name='loss3/classifier', kernel_regularizer=l2(0.0002))(pool5_drop_7x7_s1)
        loss3_classifier_act = Activation('softmax', name='prob')(loss3_classifier)
        final = Dense(num_classes, activation='softmax', kernel_initializer='he_uniform')(loss3_classifier_act)

        model = Model(inputs=InputImages, outputs=final)
        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model = model

# Tests

## Preporcessing steps for MNIST

In [8]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [9]:
#reshape data to fit model
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [10]:
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
model_simple = SimpleCNN(input_shape=(28,28,1), num_classes=10)

NameError: name 'SimpleCNN' is not defined

### Test simple

In [26]:
model_simple.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 14s 5ms/step - loss: 0.4512 - accuracy: 0.9474 - val_loss: 0.1007 - val_accuracy: 0.9702
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0664 - accuracy: 0.9801 - val_loss: 0.0992 - val_accuracy: 0.9737
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0480 - accuracy: 0.9855 - val_loss: 0.0975 - val_accuracy: 0.9756
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0399 - accuracy: 0.9880 - val_loss: 0.0964 - val_accuracy: 0.9767
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0298 - accuracy: 0.9908 - val_loss: 0.1159 - val_accuracy: 0.9756
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0271 - accuracy: 0.9921 - val_loss: 0.1157 - val_accuracy: 0.9798
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0229 - accuracy: 0.9936 - val_loss: 0.1619 - val_accuracy

In [35]:
name = model_simple.__class__.__name__
model_simple.save(name + ".h5")

In [10]:
y_pred_simple = model_simple.predict(X_test)
y_pred_simple_int = np.argmax(y_pred_simple, axis=1)
y_test_int = np.argmax(y_test, axis=1)

sum(y_pred_simple_int==y_test_int)/len(y_test_int)

313/313 [==============================] - 1s 2ms/step


0.971

### Test AlexNet

In [8]:
model_alex = AlexNet(input_shape=(28,28,1), num_classes=10)

In [9]:
model_alex.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 45s 23ms/step - loss: 0.5162 - accuracy: 0.8462 - val_loss: 0.1639 - val_accuracy: 0.9529
Epoch 2/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1890 - accuracy: 0.9552 - val_loss: 0.2114 - val_accuracy: 0.9547
Epoch 3/10
1875/1875 [==============================] - 42s 23ms/step - loss: 0.1755 - accuracy: 0.9602 - val_loss: 0.1178 - val_accuracy: 0.9708
Epoch 4/10
1875/1875 [==============================] - 42s 23ms/step - loss: 0.1409 - accuracy: 0.9691 - val_loss: 0.1126 - val_accuracy: 0.9747
Epoch 5/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1096 - accuracy: 0.9746 - val_loss: 0.1145 - val_accuracy: 0.9745
Epoch 6/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.1048 - accuracy: 0.9758 - val_loss: 0.1075 - val_accuracy: 0.9724
Epoch 7/10
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0900 - accuracy: 0.9797 - val_loss: 0.1534 -

In [10]:
name = model_alex.__class__.__name__
model_alex.save(name + ".h5")

In [15]:
y_pred_alex = model_alex.predict(X_test)
y_pred_alex_int = np.argmax(y_pred_alex, axis=1)

sum(y_pred_alex_int==y_test_int)/len(y_test_int)

313/313 [==============================] - 2s 5ms/step


0.9804

### Test SqueezeNet

In [38]:
model_sq = SqueezeNet(input_shape=(28,28,1), num_classes=10).model

In [39]:
model_sq.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 32s 16ms/step - loss: 0.5368 - accuracy: 0.8318 - val_loss: 0.1031 - val_accuracy: 0.9744
Epoch 2/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.1408 - accuracy: 0.9653 - val_loss: 0.0910 - val_accuracy: 0.9747
Epoch 3/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.1162 - accuracy: 0.9707 - val_loss: 0.0902 - val_accuracy: 0.9754
Epoch 4/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1003 - accuracy: 0.9745 - val_loss: 0.1022 - val_accuracy: 0.9736
Epoch 5/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0946 - accuracy: 0.9769 - val_loss: 0.0732 - val_accuracy: 0.9811
Epoch 6/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0865 - accuracy: 0.9784 - val_loss: 0.0835 - val_accuracy: 0.9806
Epoch 7/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0853 - accuracy: 0.9797 - val_loss: 0.0651 -

In [40]:
name = model_sq.__class__.__name__
model_sq.save(name + ".h5")

In [20]:
y_pred_sq = model_sq.predict(X_test)
y_pred_sq_int = np.argmax(y_pred_sq, axis=1)

sum(y_pred_sq_int==y_test_int)/len(y_test_int)

313/313 [==============================] - 2s 7ms/step


0.9798

### Test VGG16

In [42]:
model_vgg = VGG(input_shape=(28,28,1), num_classes=10)

In [44]:
model_vgg.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 125s 66ms/step - loss: 0.6558 - accuracy: 0.7893 - val_loss: 0.1741 - val_accuracy: 0.9572
Epoch 2/10
1875/1875 [==============================] - 123s 66ms/step - loss: 0.1384 - accuracy: 0.9676 - val_loss: 0.1318 - val_accuracy: 0.9686
Epoch 3/10
1875/1875 [==============================] - 128s 68ms/step - loss: 0.1262 - accuracy: 0.9718 - val_loss: 0.2349 - val_accuracy: 0.9462
Epoch 4/10
1875/1875 [==============================] - 123s 66ms/step - loss: 0.1241 - accuracy: 0.9715 - val_loss: 0.1274 - val_accuracy: 0.9751
Epoch 5/10
1875/1875 [==============================] - 123s 66ms/step - loss: 0.0949 - accuracy: 0.9795 - val_loss: 0.0606 - val_accuracy: 0.9842
Epoch 6/10
1875/1875 [==============================] - 128s 68ms/step - loss: 0.0908 - accuracy: 0.9808 - val_loss: 0.0676 - val_accuracy: 0.9840
Epoch 7/10
1875/1875 [==============================] - 123s 66ms/step - loss: 0.0860 - accuracy: 0.9810 - val_loss: 0

In [45]:
name = model_vgg.__class__.__name__
model_vgg.save(name +"_16_" + ".h5")

In [32]:
y_pred_vgg = model_vgg.predict(X_test)
y_pred_vgg_int = np.argmax(y_pred_vgg, axis=1)

sum(y_pred_vgg_int==y_test_int)/len(y_test_int)

313/313 [==============================] - 6s 17ms/step


0.9743

### Test VGG19

In [12]:
model_vgg_19 = VGG(input_shape=(28,28,1), num_classes=10, vgg_type=19)

In [13]:
model_vgg_19.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 158s 83ms/step - loss: 2.3341 - accuracy: 0.1119 - val_loss: 2.3011 - val_accuracy: 0.1135
Epoch 2/10
  87/1875 [>.............................] - ETA: 2:20 - loss: 2.3021 - accuracy: 0.1128

KeyboardInterrupt: 

In [ ]:
name = model_vgg_19.__class__.__name__
model_vgg_19.save(name +"_19_" + ".h5")

### Test ResNet

In [49]:
model_res = ResNet(input_shape=(28,28,1), num_classes=10).model

In [50]:
model_res.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

Epoch 1/10
1875/1875 [==============================] - 13s 6ms/step - loss: 1.2985 - accuracy: 0.5410 - val_loss: 0.4144 - val_accuracy: 0.8865
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6532 - accuracy: 0.7999 - val_loss: 0.2641 - val_accuracy: 0.9282
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.5026 - accuracy: 0.8527 - val_loss: 0.2346 - val_accuracy: 0.9383
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.4437 - accuracy: 0.8705 - val_loss: 0.1846 - val_accuracy: 0.9497
Epoch 5/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.4076 - accuracy: 0.8821 - val_loss: 0.1789 - val_accuracy: 0.9511
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3885 - accuracy: 0.8880 - val_loss: 0.1944 - val_accuracy: 0.9527
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3712 - accuracy: 0.8918 - val_loss: 0.1622 - val_ac

In [51]:
name = model_res.__class__.__name__
model_res.save(name + ".h5")

In [73]:
y_pred_res = model_res.predict(X_test)
y_pred_res_int = np.argmax(y_pred_res, axis=1)

sum(y_pred_res_int==y_test_int)/len(y_test_int)

313/313 [==============================] - 2s 5ms/step


0.924

### Test GoogleLeNet

In [15]:
model_google = GoogleLeNet(input_shape=(28,28,1), num_classes=10)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 4, 3, 64), (None, 4, 3, 128), (None, 4, 3, 32), (None, 4, 3, 32)]